In [1]:
import numpy as np
import pandas as pd
%load_ext autoreload
%autoreload 2
from config import *
import utils
import pingouin as pg
import seaborn as sns
import scipy.stats
import scprep, glob, sys
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm

# inclusion/exclusion criteria workflow

In [2]:
im=pd.read_csv(f'{NDA4}/abcd_imgincl01.txt',sep='\t',low_memory=False)
subjectkeys=im.subjectkey.unique()[1:]
include_baseline  = np.zeros_like(subjectkeys)
for i, s in enumerate(subjectkeys):
    row = im[im['subjectkey']==s]
    try:
        base = row[row['eventname']==YEAR_CODES['baseline']]['imgincl_nback_include'].item()
    except:
        base=0
    include_baseline[i]=base
subject_df = pd.DataFrame({'subjectkey':subjectkeys,
                           'imgincl_nback_include_baseline':include_baseline.astype(int),
                          'passed_step1':include_baseline.astype(int)})
print(f'After nBack exclusion: {np.sum(include_baseline.astype(int))} remaining')

After nBack exclusion: 7932 remaining


In [3]:
# Make sure not missing social environment/contrasts/CBCL
for FEAT in set((EXCLUDE_IF_MISSING + PREDICTORS_OF_INTEREST)):
    v = utils.load_variables(subjectkeys, 'baseline', FEAT, PREDICTOR_TO_VARIABLE[FEAT])
    subject_df[f'{FEAT}_baseline'] = v.astype(np.float64)

has_baseline, has_2year=np.zeros_like(subjectkeys),np.zeros_like(subjectkeys)
for i,s in enumerate(subjectkeys):
    fns = glob.glob(f'{RELEASE4_CIFTI}/{s.replace('NDAR_','')}_baseline/nBack/*')
    if len(fns)>0:
        has_baseline[i] = 1
    fns = glob.glob(f'{RELEASE4_CIFTI}/{s.replace('NDAR_','')}_2year/nBack/*')
    if len(fns)>0:
        has_2year[i] = 1
        
subject_df['has_baseline_contrasts']=has_baseline
subject_df.set_index("subjectkey", drop=True, inplace=True)
subs_no_contrasts=subject_df[subject_df['has_baseline_contrasts']==0].index

summed_values = subject_df.sum(skipna=False,axis=1).values
subs_missing = subject_df.iloc[np.where(summed_values!=summed_values)[0]].index
subs_no_nback = subject_df[subject_df['imgincl_nback_include_baseline']==0].index
vec=np.ones_like(v)
missing_features, missing_nback, missing_contrasts= 0,0,0
for i,s in enumerate(subjectkeys):
    if s in subs_missing:
        vec[i]=0
        missing_features+=1
    if s in subs_no_nback:
        vec[i]=0
        missing_nback+=1
    if s in subs_no_contrasts:
        vec[i]=0
        missing_contrasts+=1
subject_df['passed_step2']=vec
print(f'{np.sum(vec)} remaining') 
print(f'missing features={missing_features}, excluding nback={missing_nback}, missing files={missing_contrasts}')
subject_df['has_2year_contrasts']=has_2year


6354.0 remaining
missing features=1712, excluding nback=3900, missing files=2769


In [4]:
subs_after_nimg_excl_baseline=utils.format_subjects(utils.select_brain_data_subjects('baseline', 'nBack_emotion_vs_neutface'))
passed_step3=np.zeros_like(subjectkeys)
for i,s in enumerate(subjectkeys):
    if s in subs_after_nimg_excl_baseline:
        passed_step3[i]=1
subject_df['passed_step3']=passed_step3        
print(f'After QC imaging data and 1 per family, {np.sum(passed_step3)}')
# now do inclusion at baseline  timepoint overall
subject_df['include_baseline_cohort'] = subject_df['passed_step3']

After QC imaging data and 1 per family, 4732


In [5]:
vals = []
for F in ['cbcl_scr_syn_totprob_t', 'cbcl_scr_syn_internal_t', 'cbcl_scr_syn_external_t']:
    v = utils.load_variables(subjectkeys, '2year', F,  PREDICTOR_TO_VARIABLE[F])
    v = np.where(v>0,1,0) # does it exist, if not, 0
    vals.append(v)
vals=np.array(vals)

has_2year=np.zeros_like(subjectkeys)
summed=np.sum(np.array(vals), axis=0)
has_2year[np.where(summed==3)[0]]=1
im_incl = utils.load_variables(subjectkeys, '2year', ["imgincl_nback_include"],  'imgincl')
has_2year = has_2year*im_incl
subject_df['passed_step4']=has_2year
subject_df['include_longitudinal_cohort'] = subject_df['passed_step3'] + subject_df['passed_step4'] + subject_df['has_2year_contrasts']
subject_df['include_longitudinal_cohort'] = np.array([s == 3 for s in subject_df['include_longitudinal_cohort'].values]).astype(int)
print(f'{np.sum(subject_df['include_longitudinal_cohort'].values)} included at 2-year')

2371 included at 2-year


# comparison of subjects included vs not included, who have nback imaging data

In [6]:
# get data for subjects who were / werent included 
filename=f'{PROJECT_DIR}/data/overall_subjects.csv'
if os.path.exists(filename):
    df1=pd.read_csv(filename)
else:
    # subjects who had enback data but were then dropped later on in the process
    df1 = subject_df[subject_df['passed_step1']==1]
    df1 = df1[['include_longitudinal_cohort','include_baseline_cohort']]
    
    for F,V in PREDICTOR_TO_VARIABLE.items():
        v = utils.load_variables(df1.index, 'baseline', F, V)
        try:
            df1[f'{F}'] = v.astype(np.float64)
        except: 
            df1[f'{F}'] = v
df1=df1.dropna()

In [13]:
x_cols = ['cbcl_scr_syn_external_t', 'cbcl_scr_syn_internal_t', 'cbcl_scr_syn_totprob_t', 'crpbi_y_ss_caregiver',
 'fes_y_ss_fc', 'nsc_p_ss_mean_3_items',  'race_ethnicity', 'reshist_addr1_adi_perc',
'demo_prnt_ed_v2', 'demo_comb_income_v2']
x_cols_z=[f'{x}_z' for x in x_cols]

for x in x_cols:
    v = df1[x].values
    if v.dtype =='O': 
        df1.loc[df1.index, f'{x}_z'] = df1[x].values
    else: # dont try to normalize strings
        df1.loc[df1.index, f'{x}_z']=scipy.stats.zscore(v, nan_policy='omit')

In [14]:
df_excls = df1.dropna()
for x in x_cols:
    v = df_excls[x].values
    if v.dtype =='O': 
        df_excls.loc[df1.index, f'{x}_z'] = df1[x].values
    else: # dont try to normalize strings
        df_excls.loc[df_excls.index, f'{x}_z']=scipy.stats.zscore(v)
df1.loc[df1.index,'include_baseline_cohort']=df1['include_baseline_cohort'].astype(int)

In [15]:
# logistic regression for inclusion at baseline
logit_mod = sm.Logit(exog=df1[x_cols_z], endog=df1['include_baseline_cohort'])
model_res = logit_mod.fit()
model_res.summary()

Optimization terminated successfully.
         Current function value: 0.691050
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                              Logit Regression Results                             
===================================================================================
Dep. Variable:     include_baseline_cohort   No. Observations:                 6770
Model:                               Logit   Df Residuals:                     6760
Method:                                MLE   Df Model:                            9
Date:                     Tue, 13 Aug 2024   Pseudo R-squ.:                 -0.1149
Time:                             14:09:20   Log-Likelihood:                -4678.4
converged:                            True   LL-Null:                       -4196.3
Covariance Type:                 nonrobust   LLR p-value:                     1.000
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
cbcl_scr_syn_external_t_z    -0.0231      0.048     -0.480      0.631      -0.118       0.071
cbcl_scr_syn_internal_t_z     0.0600      0.048      1.250      0.211      -0.034       0.154
cbcl_scr_syn_totprob_t_z     -0.0252      0.071     -0.354      0.723      -0.165       0.114
crpbi_y_ss_caregiver_z       -0.0231      0.025     -0.919      0.358      -0.073       0.026
fes_y_ss_fc_z                 0.0039      0.026      0.154      0.877      -0.046       0.054
nsc_p_ss_mean_3_items_z      -0.0567      0.026     -2.171      0.030      -0.108      -0.006
race_ethnicity_z              0.0310      0.025      1.258      0.209      -0.017       0.079
reshist_addr1_adi_perc_z      0.0639      0.026      2.458      0.014       0.013       0.115
demo_prnt_ed_v2_z            -0.0283      0.026     -1.078      0.281      -0.080       0.023
demo_comb_income_v2_z         0.0409      0.025      1.655      0.098      -0.008       0.089
=============================================================================================
"""

In [16]:
# logistic regression for inclusion at 2 year
logit_mod = sm.Logit(exog=df1[x_cols_z], endog=df1['include_longitudinal_cohort'])
model_res = logit_mod.fit()
model_res.summary()

Optimization terminated successfully.
         Current function value: 0.692139
         Iterations 3


<class 'statsmodels.iolib.summary.Summary'>
"""
                                Logit Regression Results                               
=======================================================================================
Dep. Variable:     include_longitudinal_cohort   No. Observations:                 6770
Model:                                   Logit   Df Residuals:                     6760
Method:                                    MLE   Df Model:                            9
Date:                         Tue, 13 Aug 2024   Pseudo R-squ.:                -0.04947
Time:                                 14:09:34   Log-Likelihood:                -4685.8
converged:                                True   LL-Null:                       -4464.9
Covariance Type:                     nonrobust   LLR p-value:                     1.000
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
cbcl_scr_syn_external_t_z    -0.0851      0.048     -1.767      0.077      -0.180       0.009
cbcl_scr_syn_internal_t_z     0.0460      0.048      0.959      0.337      -0.048       0.140
cbcl_scr_syn_totprob_t_z      0.0424      0.071      0.596      0.551      -0.097       0.182
crpbi_y_ss_caregiver_z        0.0004      0.025      0.015      0.988      -0.049       0.050
fes_y_ss_fc_z                 0.0214      0.025      0.841      0.400      -0.029       0.071
nsc_p_ss_mean_3_items_z       0.0065      0.026      0.250      0.802      -0.045       0.058
race_ethnicity_z             -0.0535      0.025     -2.171      0.030      -0.102      -0.005
reshist_addr1_adi_perc_z      0.0365      0.026      1.408      0.159      -0.014       0.087
demo_prnt_ed_v2_z            -0.0029      0.025     -0.119      0.905      -0.051       0.045
demo_comb_income_v2_z        -0.0022      0.025     -0.089      0.929      -0.050       0.046
=============================================================================================
"""

# sample demographics

demo_comb_income_v2 (pdem02)<br>
``1= Less than $5,000; 2=$5,000 through $11,999; 3=$12,000 through $15,999; 
4=$16,000 through $24,999; 5=$25,000 through $34,999; 6=$35,000 through $49,999; 
7=$50,000 through $74,999; 8= $75,000 through $99,999; 9=$100,000 through $199,999; 
10=$200,000 and greater. 999 = Don't know ``<br> 
**Recoded:**
1 = < 50K (<6)<br> 
2 = < 100K (7,8)<br>
3 = > 100K (9, 10)<br> 
4 = Unknown 777/999
<br><br>
demo_prnt_ed_v2 (pdem02)<br> 
``0 = Never attended/Kindergarten only Nunca asist√É¬≠/Kinder solamente ; 1 = 1st grade 
2 = 2nd grade 3 = 3rd grade 4 = 4th grade 5 = 5th grade 6 = 6th grade 7 = 7th grade 
8 = 8th grade ; 9 = 9th grade ; 10 = 10th grade  ; 11 = 11th grade  ; 12 = 12th grade; 
13 = High school graduate  ; 14 = GED  ; 15 = Some college; 16 = Associate degree: Occupational; 
17 = Associate degree: Academic Program  ; 18 = Bachelor's degree (ex. BA; 
19 = Master's degree (ex. MA; 
20 = Professional School degree (ex. MD; 
21 = Doctoral degree (ex. PhD; 777 = Refused to answer``<br>
**Recoded**<br> 
1 = < HS (< 13)<br> 
2 = HS Diploma / GED (13,14)<br> 
3 = Some college (15,16,17)<br> 
4 = Bachelor (18)<br> 
5 = Post graduate degree (19,20,21)<br> 
6 = Unknown (777,999)<br> 
<br> 
race_ethnicity (acspw03)<br> 
``1 = White; 2 = Black; 3 = Hispanic; 4 = Asian; 5 = Other``<br> 


In [21]:
subs_included=pd.read_csv(f'{PROJECT_DIR}/data/subjects_included.csv')
df1=pd.read_csv(f'{PROJECT_DIR}/data/overall_subjects.csv')
target_cols = ['subjectkey','include_longitudinal_cohort','include_baseline_cohort','interview_age','race_ethnicity','sex','demo_comb_income_v2','demo_prnt_ed_v2','site_id_l']
df_baseline = df1[df1['include_baseline_cohort']==1]#[target_cols] 
df_longit = df1[df1['include_longitudinal_cohort']==1][target_cols] 
MERGE = lambda d1, d2 : {**d1, **d2}
recoded_income = {1:1,2:1, 3:1, 4:1, 5:1, 6:1, 7:2, 8:2, 9:3, 10:3, 777:0,999:0}
recoded_educat = MERGE({i: 1 for i in range(13)}, {13:2, 14:2, 15:3, 16:3, 17:3, 18:4, 19:5, 20:5, 21:5, 777:6, 999:6})
for tmp in [df_baseline, df_longit]:
    vals = tmp['demo_comb_income_v2'].values                                           
    recoded_inc = np.array([recoded_income[i] for i in vals])
    vals = tmp['demo_prnt_ed_v2'].values                                           
    recoded_edu = np.array([recoded_educat[i] for i in vals])
    tmp.loc[tmp.index,'recoded_income']=recoded_inc
    tmp.loc[tmp.index,'recoded_edu']=recoded_edu

/tmp/ipykernel_2799978/2816889248.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[tmp.index,'recoded_income']=recoded_inc
/tmp/ipykernel_2799978/2816889248.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp.loc[tmp.index,'recoded_edu']=recoded_edu
